# Load Data

In [4]:
from pyearth import Earth
from DLtools.Data import load_data,instant_data,check_specific_col,station_sel

import numpy as np
import pandas as pd

In [5]:
# loaddata = load_data()
# df_d = loaddata.daily()
# df_h = loaddata.hourly()
###################################
loading = instant_data()
# df,mode = loading.hourly_instant(),'hour'
df,mode = loading.daily_instant(),'day'


In [6]:
st = 'CPY012'
target,start_p,stop_p,host_path=station_sel(st,mode)
if mode =='hour': n_past,n_future = 24*7,72
elif mode =='day': n_past,n_future = 60,30

data = df[start_p:stop_p]
n_out=n_future

In [7]:
data['Day'] = data.index.dayofyear #add day
data = data.interpolate(limit=300000000,limit_direction='both').astype('float32')#interpolate neighbor first, for rest NA fill with mean() #.apply(lambda x: x.fillna(x.mean()),axis=0)

# Missing data handle

In [8]:
# # data = df_h['2013-01-01':'2017-12-31']

# # limit 80% avaliable data only
# for col in data.columns:
#     if data[col].count()<len(data[col])*.8:
#         data = data.drop(col, axis=1)
# # Fill remain with mean
# data = data.apply(lambda x: x.fillna(x.mean()),axis=0)
# data.head()

In [9]:
def to_supervise(data,target,n_out):
    data[target]=data[target].shift(-n_out)
    data = data.astype('float64').dropna()
    X = data.drop([target],axis=1)
    xlabels = list(X.columns)
    X = X.values
    y = data[target].values
    return X,y,xlabels

In [10]:
X,y,xlabels = to_supervise(data,target,n_out)

# MARS feature selection (from all stations)

In [11]:
# #scale data since it's cannot run MARS in general
# from sklearn.preprocessing import MinMaxScaler
# # SCALE
# scaler = MinMaxScaler()
# data[data.columns] = scaler.fit_transform(data[data.columns])


In [12]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #  max_terms=20,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.495728  1      0.496  0.000  0.000  
1     0       188  160   0.182981  3      0.184  0.631  0.628  
2     0       165  1256  0.137365  5      0.139  0.723  0.719  
3     0       222  1283  0.115050  7      0.118  0.768  0.763  
4     0       175  896   0.103893  9      0.107  0.790  0.785  
5     0       180  591   0.092346  11     0.096  0.814  0.807  
6     0       330  1000  0.079771  13     0.083  0.839  0.832  
7     0       293  1323  0.073921  15     0.078  0.851  0.844  
8     0       170  1110  0.069252  17     0.073  0.860  0.852  
9     0       198  620   0.064107  19     0.068  0.871  0.862  
10    0       353  217   0.060056  21     0.064  0.879  0.870  
11    0       161  1255  0.055421  23     0.060  0.888  0.879  
12    0       203

Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [13]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [14]:
print(model.summary_feature_importances(sort_by='rss')[:500])

                                   rss    gcv    nb_subsets
PAS009_wl                          0.63   0.63   0.06          
CPY011_wl                          0.09   0.09   0.03          
CPY016_wl                          0.07   0.07   0.05          
YOM008_wl                          0.06   0.06   0.03          
CPY006_wl                          0.05   0.06   0.03          
DNP022_wl                          0.02   0.02   0.03          
YOM006_wl                          0.02   0.02   0.03   


In [15]:
def toDF(rank):
    name,rss,gcv,nb_sub= list(),list(),list(),list()
    for i in range(len(rank)):
        if i%4==0:
            name.append(rank[i])
        elif i%4==1:
            rss.append(rank[i])
        elif i%4==2:
            gcv.append(rank[i])
        elif i%4==3:
            nb_sub.append(rank[i])
    data = {'feature':name,
    'rss':rss,
    'gcv':gcv,
    'nb_subset':nb_sub}
    score = pd.DataFrame(data)
    return score

In [16]:
nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]


In [14]:
rss = toDF(rss)
gcv = toDF(gcv)
nbsub = toDF(nbsub)

In [15]:
#From top 20 rss,gcv,nbsub
top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
top20

,feature,rss,gcv,nb_subset
0,CPY009_wl,0.92,0.92,0.10
1,CPY008_wl,0.03,0.03,0.05
2,NAN008_wl,0.01,0.01,0.05
3,CPY011_wl,0.01,0.01,0.02
4,DNP019_temp,0.00,0.00,0.05
5,DNP025_temp,0.00,0.00,0.05
6,GLF001_wl,0.00,0.00,0.05
7,DNP004_press,0.00,0.00,0.05
8,DIV006_wl,0.00,0.00,0.05
9,DNP010_humid,0.00,0.00,0.05


In [16]:
# top20.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]_featurelist_MAR_daily_7d.csv',index=False)

# LOOP

In [17]:
conclude_df=pd.DataFrame()
for n_out in range(1,n_out+1):
    X,y,xlabels = to_supervise(data,target,n_out)
    criteria = ('rss', 'gcv', 'nb_subsets')
    model = Earth(enable_pruning = True,
                #   max_degree=3,
                #  max_terms=20,
                minspan_alpha=.5,
                feature_importance_type=criteria,
                verbose=True)
    model.fit(X,y,xlabels=xlabels)
    nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
    gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
    rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]
    
    rss,gcv,nbsub = toDF(rss),toDF(gcv),toDF(nbsub)
    top20=pd.concat([rss,gcv,nbsub],ignore_index=True)
    top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
    top20['timestep'] = n_out

    #ADDED combine all result
    conclude_df = pd.concat([conclude_df,top20],ignore_index=True)
    if mode=='day':
        conclude_df.drop_duplicates('feature').to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_daily.csv',index=False)
        conclude_df.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_daily_Fullreport.csv',index=False)

    elif mode=='hour':
        conclude_df.drop_duplicates('feature').to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)
        conclude_df.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_hourly_Fullreport.csv',index=False)

  0.942  
36    11  17     0.03  0.036  0.944  0.940  
37    24  16     0.04  0.038  0.941  0.937  
38    47  15     0.04  0.040  0.938  0.934  
39    42  14     0.04  0.043  0.933  0.929  
40    39  13     0.05  0.049  0.923  0.919  
41    46  12     0.05  0.054  0.914  0.910  
42    38  11     0.06  0.061  0.903  0.899  
43    13  10     0.06  0.067  0.893  0.889  
44    12  9      0.08  0.087  0.861  0.856  
45    15  8      0.10  0.108  0.826  0.821  
46    19  7      0.14  0.140  0.774  0.768  
47    5   6      0.16  0.164  0.735  0.729  
48    9   5      0.19  0.196  0.680  0.674  
49    7   4      0.23  0.236  0.614  0.609  
50    35  3      0.32  0.324  0.468  0.463  
51    10  2      0.40  0.401  0.338  0.335  
52    17  1      0.60  0.603  0.000  0.000  
--------------------------------------------
Selected iteration: 13
Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
------